# compare MIN1PIPE and CaImAn

In [ ]:
import csv
from scipy import sparse
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## get dataste of min1pipe conclutions

In [ ]:
# loaction of neurons
min1pipe =[]
with open(r"C:\Users\natsu\Desktop\sotsuken\MIN1PIPE\1030conclution\location_min1pipe.csv", encoding='utf-8') as f:
    reader = csv.reader(f)
    [min1pipe.append(i[0]) for i in reader]

min1pipe[0] = 8176
min1pipe = np.asarray([int(min1pipe[i]) for i in range(len(min1pipe))])

# time spikes

filename =r"C:\Users\natsu\Desktop\sotsuken\MIN1PIPE\1030conclution\min1pipe_spike.csv"
import pandas as pd

df = pd.read_csv(filename)
#print(df)
min1pipe_spikes = df.values

## get dataste of CaImAn conclutions

In [ ]:
# loaction of neurons

caiman_location = sparse.load_npz(r"C:\Users\natsu\CaImAn\demos\notebooks\caiman_conclution.npz")
caiman = [(caiman_location.getcol(n).nonzero()[0]) for n in range(0,61)]

caiman =np.asarray(caiman)

#caimans pikes (npy.name is bas caution)
caiman_spikes = np.load(r"C:\Users\natsu\CaImAn\demos\notebooks\min1pipe_spikes.npy")

## 最近傍の計算

In [ ]:
def loc_dist(m,n):
    point=min1pipe[m]
    points=caiman[n]
    point_x = point/150 
    point_y = point%150
    points_x =[]
    points_y =[]
    D = []
    j = 0
    for i in points:
        points_x.append(int(i/150))
        points_y.append(i%150)           
        X = (points_x[j] - point_x)**2
        Y = (points_y[j] - point_y)**2 
        D.append((X + Y)**(1/2)) 
        j +=1
    
    return(m,n,np.mean(D)) 

In [ ]:
location_dist = np.asarray([[loc_dist(m,n) for n in range(len(caiman))] for m in range(len(min1pipe))])

zentai_heikin = np.mean(np.asarray([[location_dist[m][n][-1] for n in range(len(caiman))] for m in range(len(min1pipe))]))
zentai_median = np.median(np.asarray([[location_dist[m][n][-1] for n in range(len(caiman))] for m in range(len(min1pipe))]))
zentai_hensa = np.std(np.asarray([[location_dist[m][n][-1] for n in range(len(caiman))] for m in range(len(min1pipe))]))
zentai_min = np.min(np.asarray([[location_dist[m][n][-1] for n in range(len(caiman))] for m in range(len(min1pipe))]))
zentai_max = np.max(np.asarray([[location_dist[m][n][-1] for n in range(len(caiman))] for m in range(len(min1pipe))]))

print(zentai_heikin,":",zentai_median,":",zentai_hensa,":",zentai_min,";",zentai_max)

In [ ]:
#caiman側の一番近いニューロンのid
print(sortdist[:,0,1])
#caiman側と一番近い場合の距離
print(sortdist[:,0,2])

## 両方の可視化

In [ ]:
from functools import reduce

def dynamic_thresh_filter(signal, scaling):
    average = reduce(lambda a, x: a + float(x), signal) / len(signal)
    # print('average =', average)
    thresh = average * scaling
    return thresh_filter(signal, thresh)

def thresh_filter(signal, thresh):
    return [float(x) if float(x) > thresh else 0.0 for x in signal]

#Euclid距離
def euclid(a,b):
    #a,bハlist
    # [print('a_elem=', a_elem) if type(a_elem) == str else 0 for a_elem in a]
    a, b = map(lambda x: dynamic_thresh_filter(x, 10), (a,b))
    # return np.sum([np.abs(float(a[i]) - float(b[i])) for i in range(len(a))])
    return np.sqrt(np.sum([(ae - be) ** 2 for ae, be in zip(a,b)]))

def comp_with_set(m):
    #CaImAn面積候補ヲ複数ダス
    c_0 = int(sortdist[:,0,1][m])#mハMIN1PIPEノid
    c_1 = int(sortdist[:,1,1][m])
    c_2 = int(sortdist[:,2,1][m])
    
    
#------------------------------------------    
    #min1pipe_place/red
    p = min1pipe[m]
    y = int(p//150)
    x = int(p%150) 
    im = np.zeros((150,150,4),dtype=np.uint8)
    im[:,:,-1]=250#アルファチャンネルz
    im[x][y][:] = 0
    
    im[x-1][y][:] = 0
    im[x-2][y][:] = 0
    im[x+1][y][:] = 0
    im[x+2][y][:] = 0
    im[x][y-1][:] = 0
    im[x][y-2][:] = 0
    im[x][y+1][:] = 0
    im[x][y+2][:] = 0

    #caiman_place_0/green
    for q in caiman[c_0]:
        y = int(q/150)#//
        x = int(q%150) 
        im[x][y][0] = 100#red
        pil = Image.fromarray(im) 
    #caiman_place_2
    for q in caiman[c_1]:
        y = int(q/150)#//
        x = int(q%150) 
        im[x][y][1] = 100#1 green
        pil = Image.fromarray(im)
    #caiman_place_2
    for q in caiman[c_2]:
        y = int(q/150)#//
        x = int(q%150) 
        im[x][y][2] = 100#2 brue
        pil = Image.fromarray(im) 

    cs = [c_0, c_1, c_2]
    for i in range(3):
        print('{:.2f}'.format(sortdist[m,i,-2],sortdist[m,i,-1]),"■H", '{:.1f}'.format(euclid(min1pipe_spikes[m],caiman_spikes[cs[i]]) / 1000), 'K')
    # print(sortdist[m,1,-2],sortdist[m,1,-1],"■H", euclid(min1pipe_spikes[m],caiman_spikes[c_1]))
    # print(sortdist[m,2,-2],sortdist[m,2,-1],"■H", euclid(min1pipe_spikes[m],caiman_spikes[c_2]))
    
    pil = Image.fromarray(im)
    plt.imshow(pil)
    #min1pipe_spike
    plt.figure(figsize=(20,3))
    plt.plot(min1pipe_spikes[m],c="black")
    plt.show()
    #caiman_spike_0
    plt.figure(figsize=(20,3))
    plt.plot(caiman_spikes[c_0],c="red")
    plt.show()
    #caiman_spike_1
    plt.figure(figsize=(20,3))
    plt.plot(caiman_spikes[c_1],c="green")
    plt.show()
    #caiman_spike_1
    plt.figure(figsize=(20,3))
    plt.plot(caiman_spikes[c_2],c="blue")
    plt.show()

## first nerurpn of min1pipe detecting and caimans detected near neurons(3)  

In [ ]:
comp_with_set(0)

## second nerurpn of min1pipe detecting and caimans detected near neurons(3)

In [ ]:
comp_with_set(1)